In [3]:
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from selenium.webdriver.chrome.options import Options


#driver = webdriver.Chrome()

# Set up Chrome options for headless mode
chrome_options = Options()
chrome_options.add_argument('--headless')  # Run in headless mode
#hrome_options.add_argument('--disable-gpu')  # Disable GPU acceleration (sometimes necessary in headless mode)

# Initialize Chrome driver with the specified options
driver = webdriver.Chrome(options=chrome_options)

# 페이지 열기
driver.get("https://www.jobplanet.co.kr/job")

# 광고 창이 로딩될 때까지 대기 (예시로 5초 대기)
time.sleep(3)

# 버튼을 찾아 클릭
try:
    close_button = driver.find_element(By.CLASS_NAME, "ab-close-button")
    close_button.click()
except:
    # 버튼이 없는 경우 예외 처리
    pass

# 스크롤 다운
for _ in range(2000):  # 예시로 5번 스크롤
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
    time.sleep(1)  # 페이지가 로딩될 때까지 대기

# 페이지 소스 가져오기
page_source = driver.page_source

# BeautifulSoup을 사용하여 파싱
soup = BeautifulSoup(page_source, 'html.parser')

# div class="item-card"의 a 태그의 href 가져오기
item_cards = soup.find_all('div', class_='item-card')
links = [item_card.find('a').get('href') for item_card in item_cards if item_card.find('a')]

# 브라우저 종료
driver.quit()

# 각 링크에 대해 https://www.jobplanet.co.kr을 추가하여 절대 경로로 변환
absolute_links = ["https://www.jobplanet.co.kr" + link for link in links]

print(len(absolute_links))

4489


In [4]:
# CSV 파일로 저장
csv_filename = "job_details.csv"
with open(csv_filename, mode='w', encoding='utf-8', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)

    # Write header
    header = ["기업명", "직무", "주요 업무", "자격 요건", "우대 사항"]
    csv_writer.writerow(header)

    # 링크를 타고 들어가서 내가 보낸 코드 실행
    for absolute_link in absolute_links:
        # Chrome 드라이버
        # Set up Chrome options for headless mode
        chrome_options = Options()
        chrome_options.add_argument('--headless')  # Run in headless mode
        chrome_options.add_argument('--disable-gpu')  # Disable GPU acceleration (sometimes necessary in headless mode)

        # Initialize Chrome driver with the specified options
        driver = webdriver.Chrome(options=chrome_options)

        # 페이지 열기
        driver.get(absolute_link)

        # 암묵적 대기 설정 (최대 10초까지 대기)
        driver.implicitly_wait(10)

        # 기업명 추출
        company_name_element = driver.find_element(By.CLASS_NAME, "company_name")
        company_name = company_name_element.text

        # 직무 추출
        job_title_element = driver.find_element(By.CLASS_NAME, "ttl")
        job_title = job_title_element.text

        # 결과를 저장할 리스트 초기화
        result_list = [company_name, job_title]

        # "recruitment-detail__box" 내의 h3 class="recruitment-detail__tit" 찾기
        section_titles = driver.find_elements(By.CSS_SELECTOR, ".recruitment-detail__box h3.recruitment-detail__tit")

        # 각 섹션에 대한 정보 찾기
        for title in section_titles:
            section_title = title.text.strip()

            # 해당 섹션에 대한 정보 가져오기
            if section_title in ["주요 업무", "자격 요건", "우대사항"]:
                # 찾은 섹션의 부모인 .recruitment-detail__box에서 p.recruitment-detail__txt의 text 가져오기
                section_text_element = title.find_element(By.XPATH, "./following-sibling::p[@class='recruitment-detail__txt']")
                section_text = section_text_element.text.strip() if section_text_element else ""

                # 우대사항 섹션이 없으면 4번째 열은 빈 문자열로 처리
                if section_title == "우대사항" and not section_text:
                    result_list.append("")
                else:
                    result_list.append(section_text)

        # Write data in a single row
        csv_writer.writerow(result_list)

        # Chrome 드라이버 닫기
        driver.close()

print(f"Data has been written to {csv_filename}")


WebDriverException: Message: chrome not reachable
Stacktrace:
	GetHandleVerifier [0x00007FF68E5A82B2+55298]
	(No symbol) [0x00007FF68E515E02]
	(No symbol) [0x00007FF68E3D045D]
	(No symbol) [0x00007FF68E3C1DD0]
	(No symbol) [0x00007FF68E4031F4]
	(No symbol) [0x00007FF68E3FEC6C]
	(No symbol) [0x00007FF68E3FB30E]
	(No symbol) [0x00007FF68E43A198]
	(No symbol) [0x00007FF68E439CB0]
	(No symbol) [0x00007FF68E431E83]
	(No symbol) [0x00007FF68E40670A]
	(No symbol) [0x00007FF68E407964]
	GetHandleVerifier [0x00007FF68E920AAB+3694587]
	GetHandleVerifier [0x00007FF68E97728E+4048862]
	GetHandleVerifier [0x00007FF68E96F173+4015811]
	GetHandleVerifier [0x00007FF68E6447D6+695590]
	(No symbol) [0x00007FF68E520CE8]
	(No symbol) [0x00007FF68E51CF34]
	(No symbol) [0x00007FF68E51D062]
	(No symbol) [0x00007FF68E50D3A3]
	BaseThreadInitThunk [0x00007FFFB7AE257D+29]
	RtlUserThreadStart [0x00007FFFB9D0AA58+40]


In [5]:
csv_file_path = 'absolute_links.csv'

with open(csv_file_path, 'w', newline='', encoding='utf-8') as file:
    csv_writer = csv.writer(file)
    
    # 각 링크를 행으로 작성
    for link in absolute_links:
        csv_writer.writerow([link])

print(f"리스트의 길이: {len(absolute_links)}")
print(f"리스트가 '{csv_file_path}' 파일에 저장되었습니다.")

리스트의 길이: 4489
리스트가 'absolute_links.csv' 파일에 저장되었습니다.


In [10]:
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from selenium.webdriver.chrome.options import Options

# CSV 파일로 저장
csv_filename = "job_details10.csv"

# absolute_links.csv 파일에서 링크 읽어오기
with open('absolute_links.csv', 'r', encoding='utf-8') as links_file:
    absolute_links = [line.strip() for line in links_file]

# 새로운 파일에 헤더를 쓰기
with open(csv_filename, mode='w', encoding='utf-8', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)

    # Write header
    header = ["기업명", "직무", "주요 업무", "자격 요건", "우대 사항"]
    csv_writer.writerow(header)

# 새로운 파일에 데이터 추가하기
with open(csv_filename, mode='a', encoding='utf-8', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)

    # 1652번째 row부터 크롤링
    for absolute_link in absolute_links[3966:]:
        # Chrome 드라이버
        # Set up Chrome options for headless mode
        chrome_options = Options()
        chrome_options.add_argument('--headless')  # Run in headless mode
        chrome_options.add_argument('--disable-gpu')  # Disable GPU acceleration (sometimes necessary in headless mode)

        # Initialize Chrome driver with the specified options
        driver = webdriver.Chrome(options=chrome_options)

        # 페이지 열기
        driver.get(absolute_link)

        # 암묵적 대기 설정 (최대 10초까지 대기)
        driver.implicitly_wait(10)

        # 크롤링 코드
        company_name_element = driver.find_element(By.CLASS_NAME, "company_name")
        company_name = company_name_element.text

        job_title_element = driver.find_element(By.CLASS_NAME, "ttl")
        job_title = job_title_element.text

        result_list = [company_name, job_title]

        section_titles = driver.find_elements(By.CSS_SELECTOR, ".recruitment-detail__box h3.recruitment-detail__tit")

        for title in section_titles:
            section_title = title.text.strip()

            if section_title in ["주요 업무", "자격 요건", "우대사항"]:
                section_text_element = title.find_element(By.XPATH, "./following-sibling::p[@class='recruitment-detail__txt']")
                section_text = section_text_element.text.strip() if section_text_element else ""

                if section_title == "우대사항" and not section_text:
                    result_list.append("")
                else:
                    result_list.append(section_text)

        # Write data in a single row
        csv_writer.writerow(result_list)

        # Chrome 드라이버 닫기
        driver.close()

print(f"Data has been written to {csv_filename}")

Data has been written to job_details10.csv
